# STAC Geoparquet with DuckDB

## Requirements

For this example, you need to have the Python libraries `duckdb` and `stac-geoparquet` installed.

## Import of libraries

In [1]:
import json
import pystac
import duckdb
from stac_geoparquet.arrow._api import stac_table_to_items

# Install and load DuckDB spatial extension
duckdb.install_extension('spatial')
duckdb.load_extension('spatial')

# Use pygeofilter library to convert between CQL2-JSON/Text and SQL query
from pygeofilter.parsers.cql2_json import parse as json_parse
from pygeofilter.backends.duckdb import to_sql_where
from pygeofilter.util import IdempotentDict

## Define query

gte and lte describe greater than or equal and lower than or equal, respectively.

In [18]:
cql2_filter = {
  "op": "and",
  "args": [
    {
      "op": "between",
      "args": [
        {
          "property": "eo:cloud_cover"
        },
        [0, 95]
      ]
    },
    {
        "op": "s_intersects",
        "args": [
          { "property": "geometry" } ,
          {
            "type": "Polygon", # Baden-Württemberg
            "coordinates": [[
                [7.5113934084, 47.5338000528],
    			[10.4918239143, 47.5338000528],
    			[10.4918239143, 49.7913749328],
    			[7.5113934084, 49.7913749328],
    			[7.5113934084, 47.5338000528]
            ]]
          }
        ]
      }
  ]
}

## Convert CQL2-JSON filter to DuckDB SQL query

In [19]:
sql_where = to_sql_where(json_parse(cql2_filter), IdempotentDict())
print(sql_where)

(("eo:cloud_cover" BETWEEN 0 AND 95) AND ST_Intersects(ST_GeomFromWKB(geometry),ST_GeomFromHEXEWKB('0103000000010000000500000034DFB1B6AA0B1E4085B0648F53C44740509E1658D0FB244085B0648F53C44740509E1658D0FB244006A017C64BE5484034DFB1B6AA0B1E4006A017C64BE5484034DFB1B6AA0B1E4085B0648F53C44740')))


## Query data with DuckDB

In [24]:
# Define geoparquet files
geoparquet = 'https://github.com/DLR-terrabyte/pygeofilter-duckdb/raw/main/example/data-files/Sentinel-2-L2A_20190127_20190202.parquet'

# Define and execute query
# Note: union_by_name slows down the query process, but is necessary when there are properties not available in all STAC items
sql_query = f"SELECT * FROM '{geoparquet}' WHERE {sql_where}"
print(f"DuckDB SQL Query: {sql_query}\n")
db = duckdb.query(sql_query)

## Convert DuckDB result to Arrow table
table = db.fetch_arrow_table()

## Convert Arrow table to List of PyStac-Items
items = []
for item in stac_table_to_items(table): 
    item['assets'] = json.loads(item['assets'])
    items.append(pystac.Item.from_dict(item))

print(f"Result: {len(items)} items found")

DuckDB SQL Query: SELECT * FROM '/dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/geoparquet/20190127_20190202.parquet' WHERE (("eo:cloud_cover" BETWEEN 0 AND 95) AND ST_Intersects(ST_GeomFromWKB(geometry),ST_GeomFromHEXEWKB('0103000000010000000500000034DFB1B6AA0B1E4085B0648F53C44740509E1658D0FB244085B0648F53C44740509E1658D0FB244006A017C64BE5484034DFB1B6AA0B1E4006A017C64BE5484034DFB1B6AA0B1E4085B0648F53C44740')))

Result: 2 items found


## Show first item

In [25]:
items[0]

<Item id=S2A_MSIL2A_20190130T103251_N0500_R108_T32TMT_20230219T233103>

## List items as Pandas GeoDataFrame

In [26]:
import geopandas as gpd
dataframe = gpd.GeoDataFrame.from_features(items)
dataframe

,geometry,constellation,eo:cloud_cover,grid:code,instruments,mgrs:grid_square,mgrs:latitude_band,mgrs:utm_zone,platform,proj:centroid,...,s2:water_percentage,sat:orbit_state,sat:relative_orbit,terrabyte:stactools_id,view:azimuth,view:incidence_angle,view:sun_azimuth,view:sun_elevation,created,datetime
0,"POLYGON ((7.66287 47.84591, 7.68759 46.85818, ...",sentinel-2,90.261889,MGRS-32TMT,[msi],MT,T,32,sentinel-2a,"{'lat': 47.35737, 'lon': 8.40224}",...,0.797147,descending,108,S2A_T32TMT_20190130T103254_L2A,284.237729,5.870583,163.911471,23.497879,2024-05-20T16:54:16.877824Z,2019-01-30T11:32:51.024000Z
1,"POLYGON ((9.68535 47.84932, 8.99973 47.85370, ...",sentinel-2,94.104266,MGRS-32TNT,[msi],NT,T,32,sentinel-2a,"{'lat': 47.43672, 'lon': 9.24926}",...,0.000135,descending,108,S2A_T32TNT_20190130T103254_L2A,289.660418,10.358405,165.248615,23.734476,2024-05-20T16:54:16.989760Z,2019-01-30T11:32:51.024000Z


## Visualize the covered area

In [27]:
import folium
import folium.plugins as folium_plugins

map = folium.Map()
layer_control = folium.LayerControl(position='topright', collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {'fillColor': '#00000000', "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    dataframe.to_json(),
    name='Stac Item footprints',
    style_function=lambda x: style,
    control=True
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map